#### Imports

In [48]:
import getpass
import json
import numpy as np

import rhino_health as rh
from rhino_health.lib.endpoints.aimodel.aimodel_dataclass import (
    AIModel,
    AIModelCreateInput,
    ModelTypes,
    AIModelTrainInput
)
from rhino_health.lib.endpoints.endpoint import NameFilterMode

#### Loging to the RHP

In [50]:
my_project_name = "my_project"  # Replace this with your project name on Rhino Health

In [ ]:
my_username = "my_email@example.com" # Replace this with the email you use to log into Rhino Health

print("Logging In")
session = rh.login(username=my_username, password=getpass())
print("Logged In")

#### Load project

In [12]:
project = session.project.get_project_by_name(my_project_name)

#### get cohorts and schema

In [53]:
cohorts = project.search_for_cohorts_by_name(cohort_base_name,name_filter_mode=NameFilterMode.CONTAINS)
schema = cohorts[0].data_schema

# Note: There are multiple ways to retrive cohorts using the SDK, this examples relies on the cohort having similar name, such as "Site 1" and "Site 2"

#### Set image

In [54]:
my_base_ecr_uri = "my_base_ecr_uri"  # Replace this with your workgroup ecr uri
image_name = "image name"  # Replace the name of the docker image uploaded to your ecr, containing the GLM regression using nvflare code


#### Set config file

In [56]:
formula = "Yb ~ Xb + COV1 + COV2 + COV3 + COV4"
method = "NR"
glm_type = "Binomial"
config_fed_server_path = "examples/nvflare/regression-glm-coeff/config/config_fed_server.json"  # Replace this with the path to your config server file
config_fed_client_path = "examples/nvflare/regression-glm-coeff/config/config_fed_client.json"  # Replace this with the path to your config client file

with open(config_fed_server_path) as f:
    config_fed_server_input = json.loads(f.read())
with open(config_fed_client_path) as f:
    config_fed_client_input = json.loads(f.read())

# Define the formula to use for the regression
config_fed_client_input['executors'][0]['executor']['args']['formula'] = formula
config_fed_client_input['executors'][0]['executor']['args']['method'] = method
config_fed_client_input['executors'][0]['executor']['args']['glm_type'] = glm_type

#### Create AI Model

In [ ]:
aimodel = AIModelCreateInput(
    name=f'GLM Model Name', 
    description="GLM",
    input_data_schema_uids=[schema.uid],
    output_data_schema_uids=[None],
    project_uid= project.uid, 
    model_type="NVIDIA FLARE v2.3", 
    config={"container_image_uri": f"{my_base_ecr_uri}:{image_name}"} 
)

aimodel = session.aimodel.create_aimodel(aimodel)
print(f"Got aimodel '{aimodel.name}' with uid {aimodel.uid}")

run_params = AIModelTrainInput(
aimodel_uid=aimodel.uid, 
input_cohort_uids=[cohort.uid for cohort in cohorts],
one_fl_client_per_cohort=False ,
validation_cohort_uids=[], 
validation_cohorts_inference_suffix="",
timeout_seconds=600,
config_fed_server=json.dumps(config_fed_server_input), 
config_fed_client=json.dumps(config_fed_client_input), 
secrets_fed_client="",
secrets_fed_server="", 
sync=False,
)

#### Run federated model

In [ ]:
print(f"Starting to run federated training of {aimodel.name}")
model_train = session.aimodel.train_aimodel(run_params)
model_result_uid = model_train.model_result_uid
run_result = model_train.wait_for_completion()
print(f"Result status is '{run_result.status.value}', errors={run_result.result_info.get('errors') if run_result.result_info else None}")
model_output = np.load(session.model_result.get_model_params(model_result_uid), allow_pickle=True) 

#### Present results

In [ ]:
scalar_value = model_output.item()
betas = scalar_value['beta'] 
stderrs = scalar_value['fed_stderror']
print("Beta      (Stderr)\n" + "\n".join([f"{beta} ({stderr})" for beta, stderr in (zip(betas, stderrs))]))